In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import json
import statsmodels.api as sm

from filter_optimization.filter_optimization_task import extract_biomass_data

from research.weight_estimation.keypoint_utils.optics import euclidean_distance, pixel2world, depth_from_disp, convert_to_world_point


In [ ]:
df = pd.read_csv('gtsf_data.csv')

depths = []
lengths = []
for idx, row in df.iterrows():
    ann, cm = eval(row.keypoints), eval(row.camera_metadata)
#     print(ann['leftCrop'])
    wkps = pixel2world(ann['leftCrop'], ann['rightCrop'], cm)
    depth = np.median([wkp[1] for wkp in wkps.values()])
    vector = wkps['UPPER_LIP'] - wkps['TAIL_NOTCH']
    depths.append(depth)
    lengths.append(np.linalg.norm(vector))
df['depth'] = depths
df['length'] = lengths

In [ ]:
df.loc[0]

In [ ]:
plt.figure(figsize=(20, 10))

plt.scatter(df.length, df.weight)

x = np.arange(np.min(df.length), np.max(df.length), 0.01)
x2 = np.arange(np.min(df.length), np.max(df.length) + 0.05, 0.01)
y = (23.6068 * x) ** 3
plt.plot(x, y, color = 'red')
y = (22 * x2) ** 3
plt.plot(x2, y, color = 'green')
y = (25 * x) ** 3
plt.plot(x, y, color = 'green')
plt.axhline(9000)
plt.xlabel('Length (m)')
plt.ylabel('Weight (g)')
plt.title('GTSF Length vs Weight: w = (23.6068 * l) ^ 3')

In [ ]:
plt.scatter(df.length, df.weight ** (1/3))


In [ ]:
X = df.length
X = sm.add_constant(X)
model = sm.OLS(df.weight ** (1/3), X)
results = model.fit()

print(results.summary())

In [ ]:
X = df.length
model = sm.OLS(df.weight ** (1/3), X)
results = model.fit()

print(results.summary())

In [ ]:
queryCache = {}

In [ ]:
pen_id = 60
df_start_date = '2020-08-24'
df_end_date = '2020-08-26'
# pen_id = 116
# df_start_date = '2020-10-26'
# df_end_date = '2020-10-30'

if pen_id in queryCache and df_start_date in queryCache[pen_id] and df_end_date in queryCache[pen_id][df_start_date]:
    df = queryCache[pen_id][df_start_date][df_end_date]
else:
    df = extract_biomass_data(pen_id, df_start_date, df_end_date, 0)
    df.date = pd.to_datetime(df.date)

    depths = []
    lengths = []
    for idx, row in df.iterrows():
        ann, cm = row.annotation, row.camera_metadata
        wkps = pixel2world(ann['leftCrop'], ann['rightCrop'], cm)
        depth = np.median([wkp[1] for wkp in wkps.values()])
        vector = wkps['UPPER_LIP'] - wkps['TAIL_NOTCH']
        depths.append(depth)
        lengths.append(np.linalg.norm(vector))
    df['depth'] = depths
    df['length'] = lengths
    
    queryCache[pen_id] = { df_start_date: { df_end_date: df } }

In [ ]:
df = df[df.akpd_score > 0.95]

In [ ]:
plt.figure(figsize=(20, 10))

plt.scatter(df.length, df.estimated_weight_g)

x = np.arange(np.min(df.length), np.max(df.length), 0.01)
x2 = np.arange(np.min(df.length), np.max(df.length) + 0.05, 0.01)
y = (23.6068 * x) ** 3
plt.plot(x, y, color = 'red')
y = (22 * x2) ** 3
plt.plot(x2, y, color = 'green')
y = (25 * x) ** 3
plt.plot(x, y, color = 'orange')
plt.axhline(9000)
plt.xlabel('Length (m)')
plt.ylabel('Weight (g)')
plt.title('Slapoya Length vs Weight')

In [ ]:
df2 = df[((df.hour >= 7) & (df.hour <= 15))].copy()
df3 = df[((df.hour >= 7) & (df.hour <= 15))].copy()
a = df3[df3.estimated_weight_g >= 5000].estimated_weight_g
b = (23.6068 * df3[df3.estimated_weight_g < 5000].length) ** 3
print(np.mean(df2.estimated_weight_g), np.mean(df3.estimated_weight_g))

In [ ]:
np.mean((23.6068 * df2.length) ** 3)

In [ ]:
(np.mean(a) * len(a) + np.mean(b) * len(b)) / (len(a) + len(b))